In [20]:
!pip install copernicusmarine==2.2.0


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [21]:
import copernicusmarine
from copernicusmarine import CopernicusMarineService
from typing import Optional, Generator
import os

In [22]:
import pandas as pd

In [17]:
ALLOWED_SERVICES = ["omi-arco","arco-geo-series", "arco-time-series"]

In [5]:
ALLOWED_SERVICES = ["arco-geo-series", "arco-time-series"]

In [23]:
ALLOWED_SERVICES = ['original-files','wmts']

In [24]:
def filter_allowed_services(
    services: list[CopernicusMarineService],
    allowed_services: Optional[list[str]] = None,
) -> Generator[CopernicusMarineService, None, None]:
    """Yield services in the authorized list"""
    for service in services:
        if service.service_name not in allowed_services:
            continue
        else:
            yield service

In [25]:
datasets_copernicus = copernicusmarine.describe()
dataset_informations_dry_run = []
dataset_informations_download = []

Fetching catalogue 1: 100%|██████████| 2/2 [00:14<00:00,  7.46s/it]  


In [56]:
for product in datasets_copernicus.products[30:60]:
    for dataset in product.datasets:
        for version in dataset.versions:
            for part in version.parts[:1]:
                for service in filter_allowed_services(
                    part.services, ALLOWED_SERVICES):
                    try:
                        result_dry_run = copernicusmarine.get(dataset_id=dataset.dataset_id, dry_run=True,
                                                             disable_progress_bar=True)
                        dataset_informations_dry_run.append(
                            {"dataset_id": dataset.dataset_id,
                             "dataset_version": version.label,
                             "version_part": part.name,
                             "service_name": service.service_name,
                             "result_dry_run": f"{result_dry_run.status} + {result_dry_run.message}",
                             "number_of_files":result_dry_run.number_of_files_to_download,
                             "total_size_to_download":result_dry_run.total_size,
                             "first_file_to_download":result_dry_run.files[0].filename,
                             "error":False})
                    except Exception as e:
                        dataset_informations_dry_run.append(
                            {"dataset_id": dataset.dataset_id,
                             "dataset_version": version.label,
                             "version_part": part.name,
                             "service_name": service.service_name,
                             "result_dry_run": f"{result_dry_run.status} + {result_dry_run.message}",
                             "number_of_files":result_dry_run.number_of_files_to_download,
                             "total_size_to_download":result_dry_run.total_size,
                             "first_file_to_download":result_dry_run.files[0].filename,
                             "error":True})

                    try:
                        result = copernicusmarine.get(dataset_id= dataset.dataset_id, dataset_version= version.label,
                                                      dataset_part= part.name, output_directory='./',
                                                      filter=f"*{result_dry_run.files[0].filename}*",
                                                      no_directories=True)
                        dataset_informations_download.append(
                            {"dataset_id": dataset.dataset_id,
                             "dataset_version": version.label,
                             "version_part": part.name,
                             "service_name": service.service_name,
                             "result_dry_run": f"{result.status} + {result.message}",
                             "number_of_files":result.number_of_files_to_download,
                             "total_size_to_download":result.total_size,
                             "first_file_to_download":result.files[0].filename,
                             "error":False})
                        os.remove(result_dry_run.files[0].filename)
                    except Exception as e:
                        dataset_informations_download.append(
                            {"dataset_id": dataset.dataset_id,
                             "dataset_version": version.label,
                             "version_part": part.name,
                             "service_name": service.service_name,
                             "result_dry_run": f"{result.status} + {result.message}",
                             "number_of_files":result.number_of_files_to_download,
                             "total_size_to_download":result.total_size,
                             "first_file_to_download":'',
                             "error":True})

INFO - 2025-10-27T14:56:40Z - Selected dataset version: "202411"
INFO - 2025-10-27T14:56:40Z - Selected dataset part: "default"
WARNING - 2025-10-27T14:56:40Z - You are using the dataset cmems_mod_blk_phy-cur_anfc_2.5km_P1D-m, version '202411', part 'default'. This exact version and part of the dataset will be retired on the 2026-01-05T00:00:00.000Z. For more information you can check: https://marine.copernicus.eu/user-corner/product-roadmap/transition-information
INFO - 2025-10-27T14:56:40Z - Listing files on remote server...
INFO - 2025-10-27T14:56:41Z - Selected dataset version: "202411"
INFO - 2025-10-27T14:56:41Z - Selected dataset part: "default"
WARNING - 2025-10-27T14:56:41Z - You are using the dataset cmems_mod_blk_phy-cur_anfc_2.5km_P1D-m, version '202411', part 'default'. This exact version and part of the dataset will be retired on the 2026-01-05T00:00:00.000Z. For more information you can check: https://marine.copernicus.eu/user-corner/product-roadmap/transition-informatio

In [57]:
df_dry_run = pd.DataFrame(dataset_informations_dry_run)

In [58]:
df_information_download = pd.DataFrame(dataset_informations_download)

In [59]:
df_dry_run.to_csv("60_products_dry_run.csv", index=False)

In [60]:
df_information_download.to_csv("60_products_downloaded.csv", index=True)

In [44]:
! mv 30_products_downloaded 30_products_downloaded.csv